# Gromacs engines
This notebook showcases the use of the pyhton classes used to steer gromacs from python. It will only work if the gromacs executables are available (e.g. in your `$PATH` variable).

There are two main classes you will use together:
 - `aimmd.distributed.MDP`, a python class which parses a gromacs molecular dynamics parameter file (`.mdp`) and makes its content available via a dictionary-like interface
 - the `aimmd.distributed.GmxEngine` or the `aimmd.distributed.SlurmGmxEngine`, both share a common interface and are `aysnc/await` enabled python wrappers to run gromacs locally or via the SLURM workload manager, respectively

### Imports and some basic checks that everything is available

In [1]:
%%bash
# if using the module system to make gromacs and friends available:
# check that they are loaded!
#module list

In [2]:
%%bash
# unix only, check that gmx is available
which gmx

/usr/local/gromacs-2020.4/bin/gmx


In [3]:
%matplotlib inline

In [4]:
import os
import asyncio
import matplotlib.pyplot as plt
import numpy as np
import MDAnalysis as mda

In [5]:
import aimmd
import aimmd.distributed as aimmdd

Tensorflow/Keras not available


### Setup working directory and the number of gromacs simulations to run in parallel

In [6]:
n_engines = 4

#scratch_dir = "."
#scratch_dir = "/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/"
scratch_dir = "/home/think/scratch/aimmd_distributed/"
wdirs = [os.path.join(scratch_dir, f"engine_wdir{i}") for i in range(n_engines)]

for d in wdirs:
    if not os.path.isdir(d):
        os.mkdir(d)

## `aimmd.distributed.GmxEngine` (and `aimmd.distributed.SlurmGmxEngine`)
Both provide the functionality of the gromacs grompp and mdrun executables in one class, i.e. given molecular dynamics parameters and possibly an initial configuration they will setup and steer a MD run. Their interfaces differ only in the additional `sbatch_script` that the slurm engine requires at initialization time. Both engines need the gromacs exetucables to be available, specifically `gmx grompp` and `gmx mdrun`  (`gmx_mpi mdrun` for the `SlurmGmxEngine`). The `SlurmGmxEngine` naturally also must have access to the slurm executables, specifically `sbatch`, `sacct` and `scancel`. However all of these can be set either at initialization time via keyword arguments or globally as attributes to the uninitialized class.

Each engine has a `prepare()` method (which will call `grompp`) and multiple methods to then run the simulation, namely `run()`, `run_walltime()` and `run_nsteps()`. The additional `prepare_from_files()` method can be used to continue a previous MD run from given `deffnm` and `workdir` (assuming all files/parts are there), note that it will (currently) not call `grompp` again and therefore assumes that the portable run input file (`.tpr`) allows for the continuation (i.e. has no or a sufficiently large integration step limit).

In [7]:
# Let us create a list of identical engines to showcase the power of concurrent execution :)
engines = [aimmdd.GmxEngine(gro_file="gmx_infiles/conf.gro",  # required
                            top_file="gmx_infiles/topol.top",  # required
                            ndx_file="gmx_infiles/index.ndx",  # optional (can be omited or None), however naturally without an index file
                                                               # you can not reference custom groups in the .mdp-file or MDP object 
                            # limit each engine to 2 threads (the box is so small that otherwise the domain decomposition fails)
                            mdrun_extra_args="-nt 2",  # use this if your version of GMX is compiled with thread-MPI support
                            #mdrun_extra_args="-ntomp 2",  # use this for GMX without thread-MPI support
                            )
           for _ in range(n_engines)]

## `aimmd.distributed.MDP`
The `MDP` is a dictionary-like interface to a parsed gromacs molecular dynamics parameter file `.mdp` file to enable easy inspection and modification from python code. Most of the values are automatically cast to their respective types, e.g. `nsteps` will always be an `int` and `ref-t` will always be a list of `float`. The default for unknow parameters is a list of `str` to allow for the highest felxibility possible.

The class supports writing of its (possibly changed) content to a new `.mdp` file by using its `.write()` method and also knows if its content has been changed since parsing the original `.mdp` file. It even supports the (undocumented) keyformat CHARMM-GUI uses in which all `-` are replaced by `_`.

In [8]:
# one MDP object per engine, in principal we could use the same object but this way is more customizable,
# e.g. we could want to modify our setup have the engines run at a different temperatures
mdps = [aimmdd.MDP("gmx_infiles/md.mdp") for _ in range(n_engines)]

In [9]:
# lets have a look at what is inside
print("MDP has been changed since parsing: ", mdps[0].changed)
print("Parsed content:")
print("---------------")
for key, val in mdps[0].items():
    print(key, " : ", val)

MDP has been changed since parsing:  False
Parsed content:
---------------
title  :  ['test']
cpp  :  ['/lib/cpp']
include  :  ['-I../top']
define  :  []
integrator  :  ['md-vv']
dt  :  0.002
nsteps  :  -1
nstxout  :  10
nstvout  :  10
nstlog  :  10
nstenergy  :  10
nstxout-compressed  :  10
compressed-x-grps  :  ['Protein', 'SOL']
energygrps  :  ['Protein', 'SOL']
nstlist  :  10
ns-type  :  ['grid']
cutoff-scheme  :  ['Verlet']
rlist  :  1.1
coulombtype  :  ['PME']
rcoulomb  :  1.1
rvdw  :  1.1
tcoupl  :  ['Berendsen']
tc-grps  :  ['Protein', 'SOL']
tau-t  :  [0.1, 0.1]
ref-t  :  [300.0, 300.0]
Pcoupl  :  ['Berendsen']
tau-p  :  1.0
compressibility  :  [4.5e-05]
ref-p  :  [1.0]
gen-vel  :  ['no']
gen-temp  :  300.0
gen-seed  :  173529
constraints  :  ['all-bonds']


In [10]:
# lets set the xtc output frequency to 0 in all MDPs, we will use the trr anyways
# we will also increase the trr output frequency by a bit and add the `continuation` parameter
nstout = 20
for mdp in mdps:
    mdp['nstvout'] = nstout
    mdp["nstxout"] = nstout
    mdp["nstlog"] = nstout
    mdp["nstenergy"] = nstout
    mdp["nstxout-compressed"] = 0
    mdp["continuation"] = "yes"  # dont apply constraints to the initial configuration

In [11]:
# have a look again
print("MDP has been changed since parsing: ", mdps[0].changed)
print("Parsed content:")
print("---------------")
for key, val in mdps[0].items():
    print(key, " : ", val)

MDP has been changed since parsing:  True
Parsed content:
---------------
title  :  ['test']
cpp  :  ['/lib/cpp']
include  :  ['-I../top']
define  :  []
integrator  :  ['md-vv']
dt  :  0.002
nsteps  :  -1
nstxout  :  20
nstvout  :  20
nstlog  :  20
nstenergy  :  20
nstxout-compressed  :  0
compressed-x-grps  :  ['Protein', 'SOL']
energygrps  :  ['Protein', 'SOL']
nstlist  :  10
ns-type  :  ['grid']
cutoff-scheme  :  ['Verlet']
rlist  :  1.1
coulombtype  :  ['PME']
rcoulomb  :  1.1
rvdw  :  1.1
tcoupl  :  ['Berendsen']
tc-grps  :  ['Protein', 'SOL']
tau-t  :  [0.1, 0.1]
ref-t  :  [300.0, 300.0]
Pcoupl  :  ['Berendsen']
tau-p  :  1.0
compressibility  :  [4.5e-05]
ref-p  :  [1.0]
gen-vel  :  ['no']
gen-temp  :  300.0
gen-seed  :  173529
constraints  :  ['all-bonds']
continuation  :  ['yes']


### Now that we have set the molecular dynamcis parameters we can prepare a gromacs MD run.
The gromacs engines `prepare()` method will call grompp, as with grompp you can use a specific starting configuration (the grompp `-t` option) or start the structure file (`.gro`) the engine got at initialization.

### Lets prepare the first engine without a starting structure:

In [12]:
e0 = engines[0]  # get it out of the list so tab-help/completion works

In [13]:
# the prepare method is an async def function (a coroutine) and must be awaited
await e0.prepare(starting_configuration=None, workdir=wdirs[0], deffnm="test", run_config=mdps[0])

### Lets prepare all other engines at once with the same initial configuration
We can use asyncio.gather to run all coroutines concurrently, for prepare this does not make a big difference (since it is fast), but the same mechanism enables us to run all 4 gromacs engines in parallel later.

In [14]:
# create an aimmd.distributed.Trajectory of the initial configuration
init_conf = aimmdd.Trajectory(trajectory_file="gmx_infiles/conf.trr",
                              structure_file="gmx_infiles/conf.gro",
                             )

In [15]:
# and prepare the engines (the return value of prepare is None)
await asyncio.gather(*(e.prepare(starting_configuration=init_conf, workdir=wdir, deffnm="test", run_config=mdp)
                       for e, wdir, mdp in zip(engines[1:], wdirs[1:], mdps[1:])
                       )
                     )

[None, None, None]

### Now run the engines for a number of steps each.
We will first run the last engine in the list alone and then all 4 concurrently for the same number of steps to show off the power of the concurrent execution of the gromacs subprocesses.

In [16]:
import time  # import time to be able to show off ;)

In [17]:
nsteps = 100000

# run one engine and time it
start = time.time()
# the engine will return an aimmd.distributed.Trajectory with the produced trajectory (part)
traj = await engines[-1].run_steps(nsteps=nsteps, steps_per_part=True)
end = time.time()

print(f"Running one engine for {nsteps} integration steps took {round(end - start, 4)} seconds.")
print(f"The produced trajectory ({traj}) has a length of {len(traj)} frames.")
print(f"This length is the number of steps divided by the engines output frequency (={engines[-1].nstout}).")
print("Note, that we are off by plus one because the initial configuration is in the trajectory for gromacs.")
print("Note also that this is only true when explicitly passing nsteps to the `run` methods, unfortunately the real relation between frames")
print("and steps done is a bit more involved...See the docstring for `GmxEngine.steps_done` if you are brave and want to know more ;)")

Running one engine for 100000 integration steps took 82.9078 seconds.
The produced trajectory (Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir3/test.part0001.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir3/test.tpr)) has a length of 5001 frames.
This length is the number of steps divided by the engines output frequency (=20).
Note, that we are off by plus one because the initial configuration is in the trajectory for gromacs.
Note also that this is only true when explicitly passing nsteps to the `run` methods, unfortunately the real relation between frames
and steps done is a bit more involved...See the docstring for `GmxEngine.steps_done` if you are brave and want to know more ;)


/home/think/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


In [18]:
# run all engines at once and time it
start = time.time()
# Now each engine will return an aimmd.distributed.Trajectory with the produced trajectory (part)
# i.e. trajs will be a list of trajectories (in the same order as the engines in the list)
trajs = await asyncio.gather(*(e.run_steps(nsteps=nsteps, steps_per_part=True) for e in engines))
end = time.time()

print(f"Running all engines for {nsteps} integration steps took {round(end - start, 4)} seconds.")
print(f"But now we have a list of {len(trajs)} trajectories with {nsteps} steps each...")
for t in trajs:
    print(t, f" with length: {len(t)}")

/home/think/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


Running all engines for 100000 integration steps took 84.7066 seconds.
But now we have a list of 4 trajectories with 100000 steps each...
Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir0/test.part0001.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir0/test.tpr)  with length: 5001
Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir1/test.part0001.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir1/test.tpr)  with length: 5001
Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir2/test.part0001.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir2/test.tpr)  with length: 5001
Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir3/test.part0002.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir3/test.tpr)  with length: 5001


### Use `prepare_from_files` to initialize new engines and pick up where we left off with the 'old' ones.

In [19]:
# create the engines
new_engines = [aimmdd.GmxEngine(gro_file="gmx_infiles/conf.gro",
                                top_file="gmx_infiles/topol.top",
                                ndx_file="gmx_infiles/index.ndx",
                                mdrun_extra_args="-nt 2",  # use this if your version of GMX is compiled with thread-MPI support
                                #mdrun_extra_args="-ntomp 2",  # use this for GMX without thread-MPI support
                                )
               for _ in range(n_engines)]
e0 = new_engines[0]  # get one out for the autocomplete

In [20]:
# and initialize with prepare_from_files
await e0.prepare_from_files(workdir=wdirs[0], deffnm="test")

In [21]:
# and the others concurrent in one go
await asyncio.gather(*(e.prepare_from_files(workdir=wdir, deffnm="test") for e, wdir in zip(new_engines[1:], wdirs[1:])))

[None, None, None]

### Now we can do another round of MD in all engines in parallel
Note that the partnums indicate that we picked up exactly where we left of. We could additionally check using the trajectories `.last_step` and `.first_step` properties, compare and observe that the last step in the previous MD runs will be the first step in these here.

In [22]:
# run all engines at once and time it
start = time.time()
trajs = await asyncio.gather(*(e.run_steps(nsteps=nsteps, steps_per_part=True) for e in new_engines))
end = time.time()

print(f"Running all engines for {nsteps} integration steps took {end - start} seconds.")
print(f"But now we have a list of {len(trajs)} trajectories with {nsteps} steps each...")
for t in trajs:
    print(t, f" with length: {len(t)}")

/home/think/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


Running all engines for 100000 integration steps took 83.73851346969604 seconds.
But now we have a list of 4 trajectories with 100000 steps each...
Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir0/test.part0002.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir0/test.tpr)  with length: 5001
Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir1/test.part0002.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir1/test.tpr)  with length: 5001
Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir2/test.part0002.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir2/test.tpr)  with length: 5001
Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir3/test.part0003.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir3/test.tpr)  with length: 5001


### Run for specified walltime

In [23]:
walltime = 0.01 # 0.01 h = 36 s

# run all engines at once and time it
start = time.time()
trajs = await asyncio.gather(*(e.run_walltime(walltime) for e in new_engines))
end = time.time()

print(f"Running all engines for {walltime} h (={walltime*60*60} s) took {round(end - start, 4)} seconds.")

Running all engines for 0.01 h (=36.0 s) took 36.5816 seconds.


/home/think/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


### Run for specified walltime or number of steps (depending on what is reached first)
We can also use the generic `run()` method which takes one or both of the `walltime` and `nsteps` arguments, it will finish as soon as one of the conditions is fullfilled. As the `run_steps()` method it also accepts the `steps_per_part` argument making it particularly useful to run in chunks (of length walltime) but for a fixed total number of steps.

Note that we can either check if `engine.steps_done < n_steps_desired` (as we do below) or call the `engine.run(nsteps=n_steps_desired)` method until it returns `None` instead of a trajectory object, which indicates that the total number of steps done in that engine is exactly the requested number of total steps.

In [24]:
print([e.steps_done for e in new_engines])
print([e.steps_done < (max([e.steps_done for e in new_engines]) + 20000) for e in new_engines])

[242720, 242800, 242880, 349280]
[True, True, True, True]


In [25]:
walltime = 0.01 # 0.01 h = 36 s
nsteps = max([e.steps_done for e in new_engines]) + 20000

all_trajs = []
all_times = []
while any([e.steps_done < nsteps for e in new_engines]):
    # run all engines at once and time it
    start = time.time()
    trajs = await asyncio.gather(*(e.run(walltime=walltime, nsteps=nsteps, steps_per_part=False) for e in new_engines))
    end = time.time()
    all_trajs.append(trajs)
    all_times.append(end-start)

print(f"Ran for a total of {len(all_times)} loops. It took us {round(sum(all_times), 4)} seconds.")

/home/think/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "
/home/think/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "
/home/think/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index res

Ran for a total of 3 loops. It took us 106.5936 seconds.


In [26]:
# the last engine will probably already have produced a `None` instead of a trajectory in the last iteration
# (since it is some steps ahead of the others because we ran it alone at the beginning of the notebook)
all_trajs[-1]

[Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir0/test.part0006.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir0/test.tpr),
 Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir1/test.part0006.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir1/test.tpr),
 Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir2/test.part0006.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir2/test.tpr),
 None]